<a href="https://colab.research.google.com/github/MarshallPotts/CSE450-Machine-Learning/blob/main/Vehicle_CNN_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
'''

import shutil
import os

plane_dir = "/content/Airplane"
bic_dir = "/content/Bics"
bus_dir = "/content/bus"
car_dir = "/content/Cars"
heli_dir = "/content/Helicopter"
moto_dir = "/content/Motocycles"
train_dir = "/content/Train"
truck_dir = "/content/Truck"

if os.path.exists(plane_dir):
    shutil.rmtree(plane_dir)
    print(f"Deleted directory: {plane_dir}")
else:
    print(f"Directory not found: {plane_dir}")

if os.path.exists(bic_dir):
    shutil.rmtree(bic_dir)
    print(f"Deleted directory: {bic_dir}")
else:
    print(f"Directory not found: {bic_dir}")

if os.path.exists(bus_dir):
    shutil.rmtree(bus_dir)
    print(f"Deleted directory: {bus_dir}")
else:
    print(f"Directory not found: {bus_dir}")

if os.path.exists(car_dir):
    shutil.rmtree(car_dir)
    print(f"Deleted directory: {car_dir}")
else:
    print(f"Directory not found: {car_dir}")

if os.path.exists(heli_dir):
    shutil.rmtree(heli_dir)
    print(f"Deleted directory: {heli_dir}")
else:
    print(f"Directory not found: {heli_dir}")

if os.path.exists(moto_dir):
    shutil.rmtree(moto_dir)
    print(f"Deleted directory: {moto_dir}")
else:
    print(f"Directory not found: {moto_dir}")

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
    print(f"Deleted directory: {train_dir}")
else:
    print(f"Directory not found: {train_dir}")

if os.path.exists(truck_dir):
    shutil.rmtree(truck_dir)
    print(f"Deleted directory: {truck_dir}")
else:
    print(f"Directory not found: {truck_dir}")



# unzip the dataset files from google drive
!unzip "/content/drive/MyDrive/ML/archive.zip" -d "/content"


Streaming output truncated to the last 5000 lines.
  inflating: /content/Train/Train_sound_for_sleep_1316.wav  
  inflating: /content/Train/Train_sound_for_sleep_1317.wav  
  inflating: /content/Train/Train_sound_for_sleep_1318.wav  
  inflating: /content/Train/Train_sound_for_sleep_1319.wav  
  inflating: /content/Train/Train_sound_for_sleep_1320.wav  
  inflating: /content/Train/Train_sound_for_sleep_1321.wav  
  inflating: /content/Train/Train_sound_for_sleep_1322.wav  
  inflating: /content/Train/Train_sound_for_sleep_1323.wav  
  inflating: /content/Train/Train_sound_for_sleep_1324.wav  
  inflating: /content/Train/Train_sound_for_sleep_1325.wav  
  inflating: /content/Train/Train_sound_for_sleep_1326.wav  
  inflating: /content/Train/Train_sound_for_sleep_1327.wav  
  inflating: /content/Train/Train_sound_for_sleep_1328.wav  
  inflating: /content/Train/Train_sound_for_sleep_1329.wav  
  inflating: /content/Train/Train_sound_for_sleep_1330.wav  
  inflating: /content/Train/Train_

Create dataloader

In [47]:
import torchaudio
import torch
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from imblearn.over_sampling import SMOTE
import torchaudio.transforms as T

class CustomAudioDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_transform=None):
        self.audio_files = []
        self.labels = []
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform

        self._load_data() #Load the data when the object is created.




    def _load_data(self):
        for label in os.listdir(self.root_dir):
            label_dir = os.path.join(self.root_dir, label)
            if os.path.isdir(label_dir):  # Ensure it's a directory
                for audio_file in os.listdir(label_dir):
                    if audio_file.endswith(('.wav', '.mp3', '.ogg')):  # Filter audio files
                        self.audio_files.append(os.path.join(label_dir, audio_file))
                        self.labels.append(label)

        augmented_audio_files = []
        augmented_labels = []

        # Apply transformations and compute spectrograms
        for i in range(len(self.audio_files)):
            audio, sample_rate = torchaudio.load(self.audio_files[i])

            # Apply transformations to augment the dataset
            augmented_audio = self._augment_audio(audio, sample_rate)

            # Compute the Mel spectrogram frequency cepstral coefficients for each augmented audio
            for augmented in augmented_audio:
                mfcc = T.MFCC()(augmented)
                augmented_audio_files.append(mfcc)
                augmented_labels.append(self.labels[i])

        # Update the dataset with augmented data
        self.audio_files.append(augmented_audio_files)
        self.labels.append(augmented_labels)

        # Create a pandas dataframe for label access
        self.audio_labels = pd.DataFrame({'label': self.labels})

        # Update labels based on substrings in file paths
        #self._update_labels()


        # Oversample the data to balance the classes
        self._oversample_data()

    def _augment_audio(self, audio, sample_rate):
        """
        Apply a series of transformations to augment the audio data.
        Returns a list of augmented audio tensors.
        """

        augmented_audio = [audio]  # Include the original audio

        # Add random noise
        # noise = torch.randn_like(audio) * 0.005
        # augmented_audio.append(audio + noise)

        # Time shifting
        # n_fft = 400  # Example FFT size; adjust based on your spectrogram settings
        # n_freq = (n_fft // 2) + 1
        # time_shift = T.TimeStretch(n_freq=n_freq, fixed_rate=1.2)  # Stretch by 20% faster
        # stretched_audio = time_shift(audio)  # Apply the time stretch transform
        # stretched_audio = torch.abs(stretched_audio)  # Ensure non-negative values
        # augmented_audio.append(stretched_audio)

        # need to do time shift in frequency domain


        # Pitch shifting
        #pitch_shift = T.PitchShift(sample_rate, n_steps=2)  # Shift pitch by 2 semitones
        # Apply pitch shift transform
        #augmented_audio.append(pitch_shift(audio))

        # Resample to the original sample rate
        normalized_audio = T.Resample(orig_freq=sample_rate, new_freq=sample_rate)(audio)  # Resample to the original sample rate
        augmented_audio.append(normalized_audio)

        return augmented_audio


    def _update_labels(self):
        # Define a mapping of substrings to desired labels
        substring_to_label = {
            "car": "car",
            "street": "car",
            "plane": "plane",
            "air": "plane",
            "707": "plane",
            "bus": "bus",
            "train": "train",
            "truck": "truck",
            "cycl": "bicycle",
            "mixkit_bike": "bicycle",
            "helicopter": "helicopter",
            "motorbike": "motorcycle",
            "superbike": "motorcycle",
            "corvette": "car",
        }

        # Update labels based on substrings in file paths
        updated_labels = []
        for file_path in self.audio_files:
            label_found = False
            for substring, label in substring_to_label.items():
                if substring in file_path.lower():  # Case-insensitive match
                    updated_labels.append(label)
                    label_found = True
                    break
            if not label_found:
                updated_labels.append("unknown")  # Default to "unknown" if no match is found

        # Update the labels in the DataFrame
        self.audio_labels['label'] = updated_labels



    def _oversample_data(self):

      max_len = max(len(feature) for feature in self.audio_files) # Find the maximum length
      padded_features = [np.pad(feature, (0, max_len - len(feature)), 'constant') for feature in self.audio_files] # Pad to max length

      # Convert audio_files (features) and labels to numpy arrays
      features = np.array(self.audio_files)
      labels = np.array(self.audio_labels['label'])

      # Initialize SMOTE
      smote = SMOTE(random_state=42)

      # Apply SMOTE to oversample the minority classes
      features_resampled, labels_resampled = smote.fit_resample(features, labels)

      # Update the dataset with the resampled data
      self.audio_files = features_resampled.tolist()  # Convert back to list
      self.audio_labels = pd.DataFrame({'label': labels_resampled})  # Update labels



    def min_max_scale(tensor, min_val=0.0, max_val=1.0):
        """
        Scales a tensor to the range [min_val, max_val].
        """
        tensor_min = tensor.min()
        tensor_max = tensor.max()
        scaled_tensor = (tensor - tensor_min) / (tensor_max - tensor_min)  # Scale to [0, 1]
        scaled_tensor = scaled_tensor * (max_val - min_val) + min_val     # Scale to [min_val, max_val]
        return scaled_tensor


    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):


        audio_path = self.audio_labels.iloc[idx, 0]
        audio, sample_rate = torchaudio.load(audio_path)
        # Apply transformations if any
        label = self.audio_labels.iloc[idx, 1]

        if self.transform:
            audio = self.transform(audio, sample_rate)

        if self.target_transform:
            label = self.target_transform(label)

        # Apply min-max scaling to normalize the spectrogram
        audio = min_max_scale(audio, min_val=0.0, max_val=1.0)

        audio = T._transforms.ToTensor()(audio)  # Convert audio to tensor
        label = T._transforms.ToTensor()(label)  # Convert label to tensor

        return audio, label
import os
import random
import shutil
from collections import defaultdict

def stratified_split_data(root_dir, train_ratio=0.8):
    """
    Splits audio data into stratified training and testing sets.

    Args:
        root_dir (str): The root directory containing the audio subdirectories (one per label).
        train_ratio (float): The ratio of data to use for training (e.g., 0.8 for 80%).

    Creates:
        - A "training" folder with stratified data.
        - A "test" folder with stratified data.
    """
    train_dir = os.path.join(root_dir, "training")
    test_dir = os.path.join(root_dir, "test")

    # Delete the directories if they exist
    if os.path.exists(train_dir):
        shutil.rmtree(train_dir)
        print(f"Deleted directory: {train_dir}")
    else:
        print(f"Directory not found: {train_dir}")

    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)
        print(f"Deleted directory: {test_dir}")
    else:
        print(f"Directory not found: {test_dir}")


    # Create train and test directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    total_files = 0

    for label in os.listdir(root_dir):
        label_dir = os.path.join(root_dir, label)
        if os.path.isdir(label_dir) and label not in ["training", "test", "sample_data"]:  # Skip training test and sample_data dirs
            files = [f for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f))]
            random.shuffle(files)  # Shuffle files to ensure randomness

            split_index = int(len(files) * train_ratio)
            train_files = files[:split_index]
            test_files = files[split_index:]

            # Create subdirectories for the label in training and test folders
            train_label_dir = os.path.join(train_dir, label)
            test_label_dir = os.path.join(test_dir, label)
            os.makedirs(train_label_dir, exist_ok=True)
            os.makedirs(test_label_dir, exist_ok=True)

            # Move files to training folder
            for file in train_files:
                src = os.path.join(label_dir, file)
                dst = os.path.join(train_label_dir, file)
                shutil.copy2(src, dst)

            # Move files to test folder
            for file in test_files:
                src = os.path.join(label_dir, file)
                dst = os.path.join(test_label_dir, file)
                shutil.copy2(src, dst)

            # Update total file count
            total_files += len(train_files) + len(test_files)

    print(f"Data splitting completed. Total files: {total_files}")
    if total_files != 9405:
        print(f"Warning: Total file count ({total_files}) does not match the expected count (9405).")


def display_file_distribution(root_dir):
    """
    Displays the file distribution between training, test, and other subdirectories.

    Args:
        root_dir (str): The root directory containing the 'training' and 'test' subdirectories.
    """
    train_dir = os.path.join(root_dir, "training")
    test_dir = os.path.join(root_dir, "test")

    print("File Distribution:")
    print("===================")

    # Display training and test set distributions
    for dataset_type, dataset_dir in [("Training", train_dir), ("Test", test_dir)]:
        print(f"\n{dataset_type} Set:")
        total_files = 0
        for label in os.listdir(dataset_dir):
            label_dir = os.path.join(dataset_dir, label)
            if os.path.isdir(label_dir):
                num_files = len(os.listdir(label_dir))
                total_files += num_files
                print(f"  {label}: {num_files} files")
        print(f"Total {dataset_type.lower()} files: {total_files}")

    # Display files in other subdirectories
    print("\nOther Subdirectories:")
    total_other_files = 0
    for label in os.listdir(root_dir):
        label_dir = os.path.join(root_dir, label)
        if os.path.isdir(label_dir) and label not in ["training", "test"]:  # Exclude training and test folders
            num_files = len(os.listdir(label_dir))
            total_other_files += num_files
            print(f"  {label}: {num_files} files")
    print(f"Total files in other subdirectories: {total_other_files}")


#set the root directory
root_dir = "/content"  # Your original directory


Split the training and test distribution once

In [31]:
stratified_split_data(root_dir, train_ratio=0.8)  # Split data into training and testing sets

Deleted directory: /content/training
Deleted directory: /content/test
Data splitting completed. Total files: 9448


In [48]:
display_file_distribution(root_dir)  # Display file distribution between training and test sets

File Distribution:

Training Set:
  bus: 3376 files
  Airplane: 538 files
  .config: 0 files
  Cars: 184 files
  Bics: 493 files
  sample_test: 0 files
  Train: 2041 files
  Motocycles: 429 files
  .ipynb_checkpoints: 0 files
  drive: 0 files
  Helicopter: 282 files
  Truck: 212 files
Total training files: 7555

Test Set:
  bus: 845 files
  Airplane: 135 files
  .config: 0 files
  Cars: 46 files
  Bics: 124 files
  sample_test: 0 files
  Train: 511 files
  Motocycles: 108 files
  .ipynb_checkpoints: 0 files
  drive: 0 files
  Helicopter: 71 files
  Truck: 53 files
Total test files: 1893

Other Subdirectories:
  .config: 2 files
  bus: 4221 files
  sample_data: 0 files
  Airplane: 673 files
  Cars: 230 files
  Bics: 617 files
  sample_test: 3 files
  Train: 2552 files
  Motocycles: 537 files
  .ipynb_checkpoints: 0 files
  drive: 6 files
  Helicopter: 353 files
  Truck: 265 files
Total files in other subdirectories: 9459


Start Training Model

In [38]:
#copy cars folder to sample directory to use to test dataloading and other stuff
# Copy the entire Cars directory to sample_dir
shutil.copytree('/content/Cars', os.path.join(sample_dir, "Cars"))

'/content/sample_test/Cars'

In [49]:
sample_dir = '/content/sample_test'  #use the cars data to test (230files)

stratified_split_data(sample_dir, train_ratio=0.8)  # Split data into training and testing sets
display_file_distribution(sample_dir)  # Display file distribution between training and test sets



Deleted directory: /content/sample_test/training
Deleted directory: /content/sample_test/test
Data splitting completed. Total files: 230
File Distribution:

Training Set:
  Cars: 184 files
Total training files: 184

Test Set:
  Cars: 46 files
Total test files: 46

Other Subdirectories:
  Cars: 230 files
Total files in other subdirectories: 230


In [50]:
train_dataset = CustomAudioDataset(root_dir=sample_dir, transform=None, target_transform=None)
test_dataset = CustomAudioDataset(root_dir=sample_dir, transform=None, target_transform=None)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (460,) + inhomogeneous part.